In [8]:
import datetime as dt
from fmiopendata.wfs import download_stored_query
import time  # For adding delay between retries

start_date = dt.date(2023, 12, 1)  # Example start date
end_date = dt.date(2023, 12, 31)

data_list = []  # List to collect data

current_date = start_date
while current_date <= end_date:
    # Set the start time to the current date
    start_time = current_date.isoformat() + "T00:00:00Z"
    # Calculate the end time 7 days from the start time
    end_date_interval = current_date + dt.timedelta(days=6)
    # Make sure the end date does not exceed the overall end date
    if end_date_interval > end_date:
        end_date_interval = end_date
    end_time = end_date_interval.isoformat() + "T23:59:59Z"

    print(f"Downloading data from {start_time} to {end_time}")  # Optional: print statement to track progress

    max_retries = 5  # Maximum number of retries for each download attempt
    retry_delay = 5  # Delay between retries in seconds

    for attempt in range(max_retries):
        try:
            # Download data for the current 7-day interval
            obs = download_stored_query("fmi::forecast::harmonie::surface::grid",
                                        args=["bbox=18,55,35,75",  # whole Finland
                                              "starttime=" + start_time,
                                              "endtime=" + end_time])
            # If download is successful, break out of the retry loop
            break
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt < max_retries - 1:
                print(f"Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)  # Wait before retrying
            else:
                print("Max retries reached, moving to next interval")
                obs = None  # Set obs to None to handle the case where all retries fail

    if obs is not None:
        # Parse and organize the data only if the download was successful
        print(obs.data)
        for station, station_data in obs.data.items():
            times = station_data['times']
            for param, values in station_data.items():
                if param != 'times':  # Skip the 'times' key
                    for time, value in zip(times, values['values']):
                        data_list.append({'Timestamp': time, 'Station': station, param: value})

    # Move to the next 7-day interval for the next iteration
    current_date += dt.timedelta(days=7)  # Fixed to add 7 instead of 6 to avoid overlapping dates


{}
{}
{}
{}
{}


In [4]:
data_list

[]